# Summarise in Arabic

We'll now continue the example with a recipe from the Vectara website where we search our employee handbook.

See here for the REST version of this: https://docs.vectara.com/docs/api-recipes

In [1]:
%pip install -q vectara-skunk-client==0.2.7

Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging

logging.basicConfig(format='%(asctime)s %(name)-20s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

In [3]:
from vectara.core import Factory

client = Factory().build()

14:54:44 +1100 Factory              INFO:initializing builder
14:54:44 +1100 Factory              INFO:Factory will load configuration from home directory
14:54:44 +1100 HomeConfigLoader     INFO:Loading configuration from users home directory [C:\Users\david]
14:54:44 +1100 HomeConfigLoader     INFO:Loading default configuration [default]
14:54:44 +1100 HomeConfigLoader     INFO:Parsing config
14:54:44 +1100 root                 INFO:We are processing authentication type [OAuth2]
14:54:44 +1100 OAuthUtil            INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
14:54:44 +1100 OAuthUtil            INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
14:54:44 +1100 root                 INFO:initializing Client


In [4]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("01-diff-language-output")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

14:54:46 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 14:54:46.466995
14:54:46 +1100 OAuthUtil            INFO:First time requesting token, authenticating
14:54:47 +1100 OAuthUtil            INFO:Received OAuth token, will expire [12/29/2023, 15:54:47]
14:54:47 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703825687]
14:54:47 +1100 RequestUtil          INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
14:54:48 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 14:54:48.521549
14:54:48 +1100 OAuthUtil            INFO:Expiry            2023-12-29 15:54:47
14:54:48 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703825687]
14:54:48 +1100 RequestUtil          INFO:URL for operation delete-corpus is: https://api.vectara.io/v1/delete-corpus


## Create Our Demo Corpus
We now create our new corpus called, "02-employee-handbook" where we'll upload our documents for this test.

In [5]:
create_corpus_result = admin_service.create_corpus("scale-01-diff-language-output", description="Example Corpus to test different language output")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

14:54:50 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 14:54:50.829230
14:54:50 +1100 OAuthUtil            INFO:Expiry            2023-12-29 15:54:47
14:54:50 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703825687]
14:54:50 +1100 RequestUtil          INFO:URL for operation create-corpus is: https://api.vectara.io/v1/create-corpus
14:54:52 +1100 AdminService         INFO:Created new corpus with 126
14:54:52 +1100 root                 INFO:New corpus created CreateCorpusResponse(corpusId=126, status=Status(code=<StatusCode.OK: 0>, statusDetail='Corpus Created'))


## Upload Handbook to Corpus
We now upload our PDF to the corpus which will parse and then encode the document, storing the embeddings in our vector database.

In [6]:
from pathlib import Path

handbook_path = Path("../resources/vectara_employee_handbook.pdf")
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, handbook_path)

14:54:55 +1100 IndexerService       INFO:Headers: {"c": "1623270172", "o": "126"}
14:54:55 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 14:54:55.271784
14:54:55 +1100 OAuthUtil            INFO:Expiry            2023-12-29 15:54:47
14:54:55 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703825687]
vectara_employee_handbook.pdf: 52.7kB [00:03, 16.4kB/s]                                                                


UploadDocumentResponse(response=UploadDocumentResponseInner(status=None, quotaConsumed=StorageQuota(numChars='9215', numMetadataChars='4903')), document=None)

## Ask our question in Arabic
We'll now ask whether we can bring birds to the Vectara office.

In [8]:
query_service = client.query_service
#query = "هل يمكنني إحضار أي طيور إلى مكتب Vectara؟"
query = "Can I bring birds to the Vectara office?"
#response = query_service.query(query, corpus_id, response_lang="ar")
response = query_service.query(query, corpus_id, response_lang="en")

14:55:15 +1100 OAuthUtil            INFO:Current timestamp 2023-12-29 14:55:15.239215
14:55:15 +1100 OAuthUtil            INFO:Expiry            2023-12-29 15:54:47
14:55:15 +1100 OAuthUtil            INFO:Already authenticated with non-expired token, expiry is [1703825687]
14:55:15 +1100 RequestUtil          INFO:URL for operation query is: https://api.vectara.io/v1/query


## Show Result in Markdown
We'll now use the utility method "renderMarkdown" to render the Python dict

In [10]:
from vectara.util import render_markdown
from IPython.display import display, Markdown
rendered = render_markdown(query, response)
display(Markdown(rendered))


# Query: Can I bring birds to the Vectara office?

At Vectara, you can bring birds to the office as part of their avian exception policy [7.2.3]. However, common household pets like cats and dogs are not allowed [7.2.1]. Vectara is committed to wildlife conservation and supports initiatives to protect endangered species [3]. They even have an on-site mini-zoo that serves as a place of inspiration, relaxation, and education [5]. The Vectara Employee Handbook covers essential information about their pet policy [4].

 1. 7.2.3: The Avian Exception - Birds of a Feather <b>At Vectara, we believe in soaring to new heights, and what better way to embody that spirit than
by allowing birds into our workspace?</b> Yes, you read that right! *score: 0.80611074, doc-id: vectara_employee_handbook.pdf*
 2. 7.2.1: The Usual Suspects - Cats & Dogs <b>We regret to inform you that common household pets such as cats and dogs are not allowed on
the Vectara campuses.</b> Why, you ask? Because at Vectara, we're all about breaking the mold and
challenging the status quo. *score: 0.7694698, doc-id: vectara_employee_handbook.pdf*
 3. Adoption Programs <b>The Conservation Connection: Vectara is committed to wildlife conservation.</b> Our exotic pet
policy is aligned with global conservation efforts, and we actively support initiatives to protect
endangered species. *score: 0.7324146, doc-id: vectara_employee_handbook.pdf*
 4. Employee Handbook - Company Pet Policy <b>Vectara Employee Handbook</b> This employee handbook covers all the essential information to make you successful at
Vectara. *score: 0.71459365, doc-id: vectara_employee_handbook.pdf*
 5. On-site Mini Zoo <b>●   The Vectara Zoo
               ○   Our on-site mini-zoo is home to these extraordinary animals.</b> It's a place of
                       inspiration, relaxation, and education. *score: 0.71196604, doc-id: vectara_employee_handbook.pdf*
 6. Birds are not only permitted but
encouraged. <b>However, before you bring in your pet parrot or rescue raven, there are some
peculiarities to this policy that you must understand.</b> While we welcome a wide variety of winged wonders, there are some speciﬁc guidelines and
absurdities to note: *score: 0.7113421, doc-id: vectara_employee_handbook.pdf*
 7. While we adore cats and dogs, we must respectfully ask that they remain at home, where they
can continue their secret missions, philosophical ponderings, and adorable antics. <b>At Vectara,
we're forging a new path, one that requires us to leave behind the familiar and embrace the wild
unknown.</b> Rest assured, your feline and canine companions will be cheering you on from the
comfort of their favorite sunlit windows and cozy dog beds. *score: 0.7058682, doc-id: vectara_employee_handbook.pdf*
 8. The Absurd Avian Allowance: In a whimsical twist, we permit ﬂamingos on "Flamingo Fridays" to
encourage balance (both literal and metaphorical) and add a splash of fun to the oﬃce
environment. <b>Banned Birds</b> The Bird Ban: Despite our love for these speciﬁc birds, we must regretfully ban all other avian
species. *score: 0.69435954, doc-id: vectara_employee_handbook.pdf*
 9. Vectara Employee Handbook <b>This employee handbook covers all the essential information to make you successful at
Vectara.</b> 7.2 - Company Pet Policy *score: 0.69069177, doc-id: vectara_employee_handbook.pdf*
 10. Furthermore, we've consulted with leading pet psychologists, animal behaviorists, and even a
few wizards to understand the complex dynamics between humans and their four-legged
companions. <b>The conclusion? Cats and dogs are simply too mainstream for Vectara's
avant-garde environment.</b> They're playing chess while we're orchestrating interdimensional
quantum maneuvers. *score: 0.69011766, doc-id: vectara_employee_handbook.pdf*
